In [ ]:
from solana.rpc.api import Client
import hashlib
import time
import random

# Initialize Solana client
solana_client = Client("https://api.mainnet-beta.solana.com")

# QELSP Encryption and Blockchain Logging
def qrng_key(length=32):
    random.seed(time.time())  # Seed with system time for better randomness
    return ''.join(chr(random.randint(0, 255)) for _ in range(length))

def xor_strings(s1, s2):
    return ''.join(chr(ord(a) ^ ord(b)) for a, b in zip(s1, s2))

def encrypt(data, key):
    block_size = 16
    data = pad(data, block_size)
    iv = qrng_key(block_size)
    ciphertext = iv
    prev_block = iv

    for i in range(0, len(data), block_size):
        block = data[i:i + block_size]
        encrypted_block = xor_strings(block, prev_block)
        ciphertext += encrypted_block
        prev_block = xor_strings(encrypted_block, key[:block_size])
    
    return iv, ciphertext

def pad(data, block_size=16):
    padding_length = block_size - len(data) % block_size
    return data + chr(padding_length) * padding_length

# Query meme coin prices
def get_token_price(mint_address):
    try:
        # Fetch token account data from Solana blockchain
        token_data = solana_client.get_account_info(mint_address)
        if token_data["result"]["value"]:
            lamports = token_data["result"]["value"]["lamports"]
            price = lamports / 10**9  # Convert lamports to SOL
            return price
        else:
            print("Token not found.")
            return None
    except Exception as e:
        print(f"Error fetching token price: {e}")
        return None

# Blockchain Ledger
class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_block(data="Genesis Block", previous_hash="0")
    
    def create_block(self, data, previous_hash):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time.time(),
            'data': data,
            'previous_hash': previous_hash,
            'hash': self.hash_block(data, previous_hash)
        }
        self.chain.append(block)
        return block
    
    def hash_block(self, data, previous_hash):
        return hashlib.sha256((data + previous_hash).encode()).hexdigest()
    
    def print_chain(self):
        for block in self.chain:
            print(block)

blockchain = Blockchain()

def log_transaction(data):
    latest_hash = blockchain.chain[-1]['hash']
    blockchain.create_block(data=data, previous_hash=latest_hash)

# Main Function
def main():
    
    mint_address = "So11111111111111111111111111111111111111112"

    # Fetch and encrypt token price
    token_price = get_token_price(mint_address)
    if token_price is not None:
        print(f"Token Price (SOL): {token_price}")

        # Encrypt the token price
        key = qrng_key()
        iv, encrypted_price = encrypt(str(token_price), key)
        print(f"Encrypted Price: {encrypted_price.encode('utf-8').hex()}")

        # Log transaction
        log_transaction(f"Encrypted token price: {encrypted_price}")
        print("\nBlockchain Ledger:")
        blockchain.print_chain()
    else:
        print("Failed to fetch token price.")

if __name__ == "__main__":
    main()
